<a href="https://colab.research.google.com/github/PianoPlayer09/TaxFraud/blob/main/OtherModels_TaxFraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kartik2112/fraud-detection")

print("Path to dataset files:", path)

csv_file = f"{path}/fraudTrain.csv"
df = pd.read_csv(csv_file)

# Display the first few rows
print(df.head())

Path to dataset files: /kaggle/input/fraud-detection
   Unnamed: 0 trans_date_trans_time            cc_num  \
0           0   2019-01-01 00:00:18  2703186189652095   
1           1   2019-01-01 00:00:44      630423337322   
2           2   2019-01-01 00:00:51    38859492057661   
3           3   2019-01-01 00:01:16  3534093764340240   
4           4   2019-01-01 00:03:06   375534208663984   

                             merchant       category     amt      first  \
0          fraud_Rippin, Kub and Mann       misc_net    4.97   Jennifer   
1     fraud_Heller, Gutmann and Zieme    grocery_pos  107.23  Stephanie   
2                fraud_Lind-Buckridge  entertainment  220.11     Edward   
3  fraud_Kutch, Hermiston and Farrell  gas_transport   45.00     Jeremy   
4                 fraud_Keeling-Crist       misc_pos   41.96      Tyler   

      last gender                        street  ...      lat      long  \
0    Banks      F                561 Perry Cove  ...  36.0788  -81.1781   
1  

In [ ]:
import sklearn as sk

In [ ]:
# Rename columns
df.rename(columns={
    'Unnamed: 0': 'id',
    'trans_date_trans_time': 'transaction_datetime',
    'cc_num': 'card_number',
    'merchant': 'business_name',
    'category': 'income_or_deduction_type',
    'amt': 'amount',
    'first': 'taxpayer_first_name',
    'last': 'taxpayer_last_name',
    'gender': 'taxpayer_gender',
    'street': 'taxpayer_street',
    'city': 'taxpayer_city',
    'state': 'taxpayer_state',
    'zip': 'taxpayer_zip',
    'lat': 'taxpayer_lat',
    'long': 'taxpayer_long',
    'city_pop': 'taxpayer_city_population',
    'job': 'taxpayer_job',
    'dob': 'taxpayer_dob',
    'trans_num': 'deduction_or_income_id',
    'unix_time': 'unix_filing_time',
    'merch_lat': 'business_lat',
    'merch_long': 'business_long',
    'is_fraud': 'is_fraudulent'
}, inplace=True)

print("Renamed columns:", df.columns)

Renamed columns: Index(['id', 'transaction_datetime', 'card_number', 'business_name',
       'income_or_deduction_type', 'amount', 'taxpayer_first_name',
       'taxpayer_last_name', 'taxpayer_gender', 'taxpayer_street',
       'taxpayer_city', 'taxpayer_state', 'taxpayer_zip', 'taxpayer_lat',
       'taxpayer_long', 'taxpayer_city_population', 'taxpayer_job',
       'taxpayer_dob', 'deduction_or_income_id', 'unix_filing_time',
       'business_lat', 'business_long', 'is_fraudulent'],
      dtype='object')


In [ ]:
fraud_counts = df['is_fraudulent'].value_counts()
print("Fraudulent and Non-Fraudulent Counts:\n", fraud_counts)

Fraudulent and Non-Fraudulent Counts:
 is_fraudulent
0    1289169
1       7506
Name: count, dtype: int64


In [ ]:
from scipy.stats import zscore

# ✅ Identify all numeric columns
numeric_columns = df.select_dtypes(include=['number']).columns

# ✅ Compute Z-scores for all numeric columns
df['z_score'] = zscore(df['amount'])  # If you only want 'amount'
df[numeric_columns] = df[numeric_columns].apply(zscore)

# ✅ Remove rows where the Z-score is above 3 for 'amount' (outlier removal)
df_cleaned = df[df['z_score'].abs() < 3].drop(columns=['z_score'])

# ✅ Save the cleaned dataset
df_cleaned.to_csv("tax_fraud_zscore_cleaned.csv", index=False)

# ✅ Print summary
print(f"✅ Removed {len(df) - len(df_cleaned)} outliers from the dataset.")
print(df_cleaned.describe())

✅ Removed 12738 outliers from the dataset.
                 id   card_number        amount  taxpayer_zip  taxpayer_lat  \
count  1.283937e+06  1.283937e+06  1.283937e+06  1.283937e+06  1.283937e+06   
mean   3.367660e-05 -4.498550e-05 -6.150735e-02 -3.686842e-05  3.771723e-04   
std    9.998431e-01  9.999373e-01  4.192291e-01  9.998778e-01  9.998825e-01   
min   -1.732049e+00 -3.187577e-01 -4.325897e-01 -1.767869e+00 -3.646814e+00   
25%   -8.657589e-01 -3.186202e-01 -3.791951e-01 -8.390099e-01 -7.621886e-01   
50%   -1.202187e-04 -3.160672e-01 -1.466544e-01 -2.330220e-02  1.608963e-01   
75%    8.659032e-01 -3.152108e-01  7.091598e-02  8.630553e-01  6.703917e-01   
max    1.732049e+00  3.495671e+00  2.999819e+00  1.895732e+00  5.547035e+00   

       taxpayer_long  taxpayer_city_population  unix_filing_time  \
count   1.283937e+06              1.283937e+06      1.283937e+06   
mean   -1.504956e-05             -5.976107e-04      4.860036e-05   
std     9.998161e-01              9.98782

In [ ]:
# ✅ Convert 'transaction_datetime' to a numeric UNIX timestamp
df['transaction_datetime'] = pd.to_datetime(df['transaction_datetime'])
df['transaction_unix'] = df['transaction_datetime'].astype(int) / 10**9  # Convert to seconds
df.drop(columns=['transaction_datetime'], inplace=True)  # Drop original datetime column

In [ ]:
from sklearn.preprocessing import LabelEncoder

# ✅ Identify categorical columns (non-numeric)
categorical_cols = df.select_dtypes(include=['object']).columns

# ✅ Apply Label Encoding
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

In [ ]:
# ✅ Fix the `is_fraudulent` column before SMOTE
df_cleaned['is_fraudulent'] = df_cleaned['is_fraudulent'].round().astype(int)

# ✅ Verify unique values
print("Unique values after correction:", df_cleaned['is_fraudulent'].unique())

Unique values after correction: [ 0 13]


In [ ]:
df_cleaned['is_fraudulent'] = df_cleaned['is_fraudulent'].replace({13: 1})  # Fix incorrect labels

In [ ]:
print("Unique values before SMOTE:", df_cleaned['is_fraudulent'].unique())

Unique values before SMOTE: [0 1]


In [ ]:
from imblearn.over_sampling import SMOTE

# ✅ Define Features (X) and Target (y)
X = df.drop(columns=['is_fraudulent'])  # Features
y = df['is_fraudulent']  # Target

In [ ]:
# ✅ Identify Non-Numeric Columns
non_numeric_cols = X.select_dtypes(include=['object']).columns
print("Non-Numeric Columns:", non_numeric_cols.tolist())

Non-Numeric Columns: []


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Apply Label Encoding to Non-Numeric Columns
for col in non_numeric_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

print("✅ Successfully converted all categorical data into numeric format!")

✅ Successfully converted all categorical data into numeric format!


In [ ]:
# ✅ Ensure y only contains 0 and 1
y = (y > 0).astype(int)

print("After Fix, Unique `y` values:", y.unique())  # Should be [0, 1]

After Fix, Unique `y` values: [0 1]


In [ ]:
print("Before SMOTE, Unique `y` values:", y.unique())  # Should output [0, 1]
print(y.value_counts())  # Check count of each class

Before SMOTE, Unique `y` values: [0 1]
is_fraudulent
0    1289169
1       7506
Name: count, dtype: int64


In [ ]:
# ✅ Apply SMOTE with Controlled Fraud Sampling
target_fraud_cases = 100000  # Define the exact number of fraudulent samples we want

# Define Sampling Strategy
sampling_strategy = {0: y.value_counts()[0], 1: y.value_counts()[1] + target_fraud_cases}

# ✅ Apply SMOTE
smote = SMOTE(sampling_strategy=sampling_strategy, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# ✅ Convert back to DataFrame
df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
df_resampled['is_fraudulent'] = y_resampled  # Add target column back

# ✅ Save the Balanced Dataset
df_resampled.to_csv("tax_fraud_balanced.csv", index=False)

# ✅ Print Summary
print(f"✅ Original dataset shape: {df.shape}")
print(f"✅ Resampled dataset shape: {df_resampled.shape}")
print(df_resampled['is_fraudulent'].value_counts())  # Check class balance

✅ Original dataset shape: (1296675, 24)
✅ Resampled dataset shape: (1396675, 24)
is_fraudulent
0    1289169
1     107506
Name: count, dtype: int64


In [ ]:
# ✅ Reduce dataset size
import pandas as pd

# ✅ Define number of samples for each class
n_samples_per_class = 50_000  # Adjust this based on your system capacity

# ✅ Separate fraud and non-fraud cases
df_fraud = df_resampled[df_resampled['is_fraudulent'] == 1].sample(n=n_samples_per_class, random_state=42)
df_non_fraud = df_resampled[df_resampled['is_fraudulent'] == 0].sample(n=n_samples_per_class, random_state=42)

# ✅ Combine both classes
df_small = pd.concat([df_fraud, df_non_fraud]).sample(frac=1, random_state=42)  # Shuffle

# ✅ Print new dataset size
print(f"✅ Reduced dataset shape: {df_small.shape}")

# ✅ Save smaller dataset for future use
df_small.to_csv("tax_fraud_small.csv", index=False)

✅ Reduced dataset shape: (100000, 24)


In [ ]:
from sklearn.model_selection import train_test_split

# ✅ Split Data (80% Train, 20% Test)
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# ✅ Print Shapes
print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")

Training set shape: (1117340, 23)
Testing set shape: (279335, 23)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# ✅ Identify Numeric Columns
numeric_cols = X_train.select_dtypes(include=['number']).columns

# ✅ Apply MinMax Scaling (0-1 Normalization)
scaler = MinMaxScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])  # Use same scaler

print("✅ Numeric Features Normalized!")

✅ Numeric Features Normalized!


In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# ✅ Identify Categorical Columns
categorical_cols = X_train.select_dtypes(include=['object']).columns

# ✅ Apply Ordinal Encoding
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X_train[categorical_cols] = encoder.fit_transform(X_train[categorical_cols])
X_test[categorical_cols] = encoder.transform(X_test[categorical_cols])  # Use same encoder

print("✅ Categorical Features Encoded!")

✅ Categorical Features Encoded!


In [ ]:
import torch

# ✅ Convert to Tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

print("✅ Data Converted to PyTorch Tensors!")

✅ Data Converted to PyTorch Tensors!


In [ ]:
df.describe()

,id,card_number,business_name,income_or_deduction_type,amount,taxpayer_first_name,taxpayer_last_name,taxpayer_gender,taxpayer_street,taxpayer_city,...,taxpayer_city_population,taxpayer_job,taxpayer_dob,deduction_or_income_id,unix_filing_time,business_lat,business_long,is_fraudulent,z_score,transaction_unix
count,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,...,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06
mean,-8.066161e-18,9.494726e-17,3.428585e+02,6.227787e+00,-9.447052e-18,1.802624e+02,2.505209e+02,4.525513e-01,4.880344e+02,4.452633e+02,...,-1.350753e-18,2.511734e+02,5.342329e+02,6.483370e+05,-2.571352e-15,8.127753e-16,9.710216e-16,9.775836e-18,-9.447052e-18,1.570107e+09
std,1.000000e+00,1.000000e+00,2.009519e+02,3.913443e+00,1.000000e+00,9.753379e+01,1.366741e+02,4.977437e-01,2.800608e+02,2.586001e+02,...,1.000000e+00,1.401094e+02,2.674801e+02,3.743180e+05,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.285545e+07
min,-1.732049e+00,-3.187577e-01,0.000000e+00,0.000000e+00,-4.325897e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,-2.940871e-01,0.000000e+00,0.000000e+00,0.000000e+00,-1.858664e+00,-3.818076e+00,-5.551108e+00,-7.630436e-02,-4.325897e-01,1.546301e+09
25%,-8.660247e-01,-3.186202e-01,1.650000e+02,3.000000e+00,-3.786337e-01,9.400000e+01,1.380000e+02,0.000000e+00,2.520000e+02,2.240000e+02,...,-2.917027e-01,1.310000e+02,3.170000e+02,3.241685e+05,-8.171226e-01,-7.444080e-01,-4.844070e-01,-7.630436e-02,-3.786337e-01,1.559589e+09
50%,0.000000e+00,-3.160672e-01,3.460000e+02,6.000000e+00,-1.424127e-01,1.830000e+02,2.520000e+02,0.000000e+00,4.850000e+02,4.390000e+02,...,-2.860297e-01,2.510000e+02,5.650000e+02,6.483370e+05,4.758308e-04,1.621089e-01,2.024585e-01,-7.630436e-02,-1.424127e-01,1.570088e+09
75%,8.660247e-01,-3.152108e-01,5.140000e+02,1.000000e+01,7.977349e-02,2.570000e+02,3.700000e+02,1.000000e+00,7.200000e+02,6.770000e+02,...,-2.268423e-01,3.740000e+02,7.560000e+02,9.725055e+05,7.897767e-01,6.692698e-01,7.254089e-01,-7.630436e-02,7.977349e-02,1.580224e+09
max,1.732049e+00,3.495671e+00,6.920000e+02,1.300000e+01,1.801352e+02,3.510000e+02,4.800000e+02,1.000000e+00,9.820000e+02,8.930000e+02,...,9.332066e+00,4.930000e+02,9.670000e+02,1.296674e+06,1.757862e+00,5.670086e+00,1.690176e+00,1.310541e+01,1.801352e+02,1.592742e+09


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Layer, Dropout

# Custom Linear Layer
class LinearLayer(Layer):
    def __init__(self, units, **kwargs):
        super(LinearLayer, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='zeros',
                                 trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

# Define Neural Network Model using only Linear Layers
model = Sequential([
    LinearLayer(128, input_shape=(X_train.shape[1],)),  # Linear transformation
    Dropout(0.3),
    LinearLayer(64),
    Dropout(0.3),
    LinearLayer(32),
    LinearLayer(1)  # No activation (purely linear)
])

from sklearn.metrics import accuracy_score

# Get predictions
y_pred = model.predict(X_test)
y_pred_binary = (y_pred >= 0.5).astype(int)  # Convert to binary labels

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_binary)
print(f'Test Accuracy: {accuracy:.4f}')

# Compile Model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])  # Using MSE for purely linear regression

# Train Model
history = model.fit(X_train, y_train, epochs=5, batch_size=24, validation_data=(X_test, y_test))

# Evaluate Model
loss, metric = model.evaluate(X_test, y_test)
print(f'Test Metric: {metric:.4f}')


<ipython-input-24-32aa2915ee6a>:8: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super(LinearLayer, self).__init__(**kwargs)


8730/8730 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step
Test Accuracy: 0.9230
Epoch 1/5
46556/46556 ━━━━━━━━━━━━━━━━━━━━ 140s 3ms/step - loss: 0.0536 - mae: 0.1184 - val_loss: 0.0493 - val_mae: 0.0978
Epoch 2/5
46556/46556 ━━━━━━━━━━━━━━━━━━━━ 163s 3ms/step - loss: 0.0522 - mae: 0.1133 - val_loss: 0.0488 - val_mae: 0.0982
Epoch 3/5
46556/46556 ━━━━━━━━━━━━━━━━━━━━ 137s 3ms/step - loss: 0.0523 - mae: 0.1134 - val_loss: 0.0550 - val_mae: 0.1595
Epoch 4/5
46556/46556 ━━━━━━━━━━━━━━━━━━━━ 141s 3ms/step - loss: 0.0526 - mae: 0.1137 - val_loss: 0.0534 - val_mae: 0.0931
Epoch 5/5
46556/46556 ━━━━━━━━━━━━━━━━━━━━ 141s 3ms/step - loss: 0.0529 - mae: 0.1128 - val_loss: 0.0491 - val_mae: 0.0918
8730/8730 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - loss: 0.0498 - mae: 0.0917
Test Metric: 0.0918


In [ ]:
# Get predictions
y_pred = model.predict(X_test)
y_pred_binary = (y_pred >= 0.5).astype(int)  # Convert to binary labels

accuracy = accuracy_score(y_test, y_pred_binary)
print(f'Test Accuracy: {accuracy:.4f}')

8730/8730 ━━━━━━━━━━━━━━━━━━━━ 29s 3ms/step
Test Accuracy: 0.9432
